<a href="https://colab.research.google.com/github/IvoandricGBC/Project-MLI-GBC/blob/main/distance_estimation_env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cuda


In [2]:
# ─── MÓDULO B: CARGA, CONFIGURACIÓN Y SPLIT ───


# 1. Semilla global para reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# 2. Definición de rutas
env_name = 'distance_estimation'
base_dir  = f"/content/drive/MyDrive/Colab Notebooks/{env_name}"
csv_path  = os.path.join(base_dir, 'labels.csv')
img_dir   = os.path.join(base_dir, 'images')

# 3. Carga de datos de metadatos
df = pd.read_csv(csv_path)
# Forzamos que ID sea string para todo el pipeline
df['ID'] = df['ID'].astype(str)

assert 'class' in df.columns, "Falta la columna 'class' en labels.csv"

# 4. Split estratificado en train/val
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=SEED,
    stratify=df['class']  # conserva proporción de clases
)

# 5. Feedback por consola
print(f"Train shape: {train_df.shape}, Val shape: {val_df.shape}")
print("Distribución de clases en train:\n", train_df['class'].value_counts(normalize=True))
print("Distribución de clases en val:\n",   val_df  ['class'].value_counts(normalize=True))

# 6. Opcional: inspección rápida de algunas rutas
print(f"Ejemplo de imagen disponible en: {os.path.join(img_dir, train_df.iloc[0]['ID'] + '.png')}")


Train shape: (400, 6), Val shape: (100, 6)
Distribución de clases en train:
 class
1    0.5275
0    0.4725
Name: proportion, dtype: float64
Distribución de clases en val:
 class
1    0.53
0    0.47
Name: proportion, dtype: float64
Ejemplo de imagen disponible en: /content/drive/MyDrive/Colab Notebooks/distance_estimation/images/164.png


In [3]:
# ─── MÓDULO C: DATASET Y TRANSFORMACIONES ───

# ─── MÓDULO C: DATASET Y TRANSFORMACIONES ───

import os
import pandas as pd
import numpy as np
import torch
from PIL import Image
from sklearn.preprocessing import StandardScaler
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# 1. Columnas de metadata
metadata_cols = ['Width', 'location']

# 2. Escalado de metadatos (solo sobre train ¡IMPORTANTE!)
scaler = StandardScaler()

# Ajuste y transformación en train
scaled_train = scaler.fit_transform(train_df[metadata_cols].astype(float))
train_df[metadata_cols] = pd.DataFrame(
    scaled_train,
    columns=metadata_cols,
    index=train_df.index
)

# Transformación en val
scaled_val = scaler.transform(val_df[metadata_cols].astype(float))
val_df[metadata_cols] = pd.DataFrame(
    scaled_val,
    columns=metadata_cols,
    index=val_df.index
)

# 3. Normalización del label (min-max) basada en train_df
min_l, max_l = train_df['label'].min(), train_df['label'].max()
train_df['label_norm'] = (train_df['label'] - min_l) / (max_l - min_l)
val_df['label_norm']   = (val_df['label']   - min_l) / (max_l - min_l)

# 4. Transformaciones de imagen
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])
val_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# 5. Dataset personalizado
class DistanceDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, f"{row['ID']}.png")
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        meta_vals = row[metadata_cols].values.astype(np.float32)
        meta = torch.from_numpy(meta_vals)
        label = torch.tensor(row['label_norm'], dtype=torch.float32)
        return img, meta, label

# 6. DataLoaders
batch_size = 32
train_ds = DistanceDataset(train_df, img_dir, transform=train_transforms)
val_ds   = DistanceDataset(val_df,   img_dir, transform=val_transforms)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                          num_workers=4, pin_memory=True, persistent_workers=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False,
                          num_workers=4, pin_memory=True, persistent_workers=True)

print(f"[+] DataLoaders OK — {len(train_ds)} imgs en train, {len(val_ds)} en val")


[+] DataLoaders OK — 400 imgs en train, 100 en val


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [4]:
# ─── MÓDULO D: DATALOADERS AVANZADOS ───

import torch
from torch.utils.data import DataLoader

# Parámetros comunes
batch_size = 32      # o ajústalo según tu memoria disponible
num_workers = 4      # idealmente = número de CPUs disponibles
prefetch_factor = 2  # cuántos batches prefetch por worker
drop_last = True     # para mantener consistencia de tamaño de batch

# Generador para reproducibilidad en el shuffle
g = torch.Generator()
g.manual_seed(SEED)

# 1. Train loader con shuffling reproducible
train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=prefetch_factor,
    drop_last=drop_last,
    generator=g
)

# 2. Val loader sin shuffle
val_loader = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=prefetch_factor,
    drop_last=False
)

# 3. (Opcional) Sampler ponderado si hubiera desbalance
# from torch.utils.data import WeightedRandomSampler
# class_counts = train_df['class'].value_counts().to_dict()
# weights = [1.0 / class_counts[c] for c in train_df['class']]
# sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)
# train_loader = DataLoader(train_ds, batch_size=batch_size, sampler=sampler, ...)

print(f"[+] Train loader: {len(train_loader)} batches, Val loader: {len(val_loader)} batches")


[+] Train loader: 12 batches, Val loader: 4 batches


In [5]:
# ─── MÓDULO E: ARQUITECTURA DEL MODELO ───

import torch
import torch.nn as nn
from torchvision import models

class DistanceModel(nn.Module):
    def __init__(self, num_meta_feat, head_sizes=[256, 128], backbone_name='resnet50', pretrained=True):
        super().__init__()
        # 1. Backbone CNN
        if backbone_name == 'resnet50':
            self.backbone = models.resnet50(pretrained=pretrained)
        elif backbone_name == 'resnet34':
            self.backbone = models.resnet34(pretrained=pretrained)
        else:
            raise ValueError(f"Backbone {backbone_name} no soportado")

        # 2. Congelar todas las capas excepto layer4 (freezing progresivo)
        for name, param in self.backbone.named_parameters():
            param.requires_grad = False
            if 'layer4' in name:
                param.requires_grad = True

        # 3. Sustituir la cabeza final de la CNN
        cnn_feats = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()   # deja pasar los features puros

        # 4. MLP para metadata
        self.meta_net = nn.Sequential(
            nn.Linear(num_meta_feat, head_sizes[0]),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(head_sizes[0], head_sizes[1]),
            nn.ReLU(),
            nn.Dropout(0.2),
        )

        # 5. Cabeza combinada (imagen + metadata)
        self.head = nn.Sequential(
            nn.Linear(cnn_feats + head_sizes[1], 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)  # salida regresión
        )

    def forward(self, img, meta):
        # Extraer features de la imagen
        x_img = self.backbone(img)           # [batch, cnn_feats]
        # Procesar metadata
        x_meta = self.meta_net(meta)         # [batch, head_sizes[1]]
        # Concatenar
        x = torch.cat([x_img, x_meta], dim=1)
        # Cabeza final
        return self.head(x)

# ─ Instanciación ─
num_meta = len(metadata_cols)  # p.ej. ['Width','location'] → 2
model = DistanceModel(num_meta_feat=num_meta, backbone_name='resnet50').to(device)

# ─── MÓDULO E: ARQUITECTURA DEL MODELO ───
# (todo lo que ya tienes, incluida la clase DistanceModel y la instanciación)
model = DistanceModel(num_meta_feat=num_meta, backbone_name='resnet50').to(device)

# ─── Configuración del optimizador con LRs diferenciados ───
import torch.optim as optim

optimizer = optim.Adam([
    # 1) Ajuste fino suave de layer4 (backbone)
    {'params': model.backbone.layer4.parameters(), 'lr': 1e-4},
    # 2) Aprendizaje más rápido de la MLP de metadata
    {'params': model.meta_net.parameters(),           'lr': 1e-3},
    # 3) Cabeza combinada imagen+metadata
    {'params': model.head.parameters(),               'lr': 1e-3},
])



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# ─── MÓDULO F: BUCLE DE ENTRENAMIENTO CON MIXED PRECISION (torch.amp) Y EARLY STOPPING ───

import numpy as np
import torch
import torch.nn as nn
from torch.amp import autocast, GradScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Configuración básica
device      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_str  = 'cuda' if torch.cuda.is_available() else 'cpu'

loss_fn      = nn.MSELoss()
scaler      = GradScaler(device_str)
num_epochs   = 50
patience     = 5
best_val_loss = float('inf')
stale_epochs  = 0

# 2. Loop de epochs
for epoch in range(1, num_epochs + 1):
    # ––– Entrenamiento –––
    model.train()
    train_losses = []
    for imgs, metas, labels in train_loader:
        imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast(device_str):
            preds = model(imgs, metas).squeeze(1)
            loss  = loss_fn(preds, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_losses.append(loss.item())

    train_loss = np.mean(train_losses)

    # ––– Validación –––
    model.eval()
    val_losses  = []
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, metas, labels in val_loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            with autocast(device_str):
                preds = model(imgs, metas).squeeze(1)
                loss  = loss_fn(preds, labels)
            val_losses.append(loss.item())
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

    val_loss = np.mean(val_losses)
    y_pred   = torch.cat(all_preds).numpy()
    y_true   = torch.cat(all_labels).numpy()
    val_mae  = mean_absolute_error(y_true, y_pred)
    val_r2   = r2_score(y_true, y_pred)

    print(
        f"Epoch {epoch:02d} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss:   {val_loss:.4f} | "
        f"Val MAE:    {val_mae:.4f} | "
        f"Val R²:     {val_r2:.4f}"
    )

    # ––– Early Stopping y checkpoint –––
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        stale_epochs  = 0
        torch.save({
            'epoch':        epoch,
            'model_state':  model.state_dict(),
            'opt_state':    optimizer.state_dict(),
            'scaler_state': scaler.state_dict()
        }, 'best_checkpoint.pt')
        print("  → Mejor modelo guardado")
    else:
        stale_epochs += 1
        if stale_epochs >= patience:
            print(f"Stopping early! No mejora en {patience} epochs.")
            break


Epoch 01 | Train Loss: 0.3705 | Val Loss:   0.0407 | Val MAE:    0.1631 | Val R²:     0.3242
  → Mejor modelo guardado
Epoch 02 | Train Loss: 0.0747 | Val Loss:   0.0951 | Val MAE:    0.2661 | Val R²:     -0.6762
Epoch 03 | Train Loss: 0.0621 | Val Loss:   0.0247 | Val MAE:    0.1415 | Val R²:     0.5064
  → Mejor modelo guardado
Epoch 04 | Train Loss: 0.0394 | Val Loss:   0.0180 | Val MAE:    0.1158 | Val R²:     0.6347
  → Mejor modelo guardado
Epoch 05 | Train Loss: 0.0369 | Val Loss:   0.0257 | Val MAE:    0.1392 | Val R²:     0.4899
Epoch 06 | Train Loss: 0.0310 | Val Loss:   0.0129 | Val MAE:    0.0974 | Val R²:     0.7414
  → Mejor modelo guardado
Epoch 07 | Train Loss: 0.0298 | Val Loss:   0.0143 | Val MAE:    0.0875 | Val R²:     0.7878
Epoch 08 | Train Loss: 0.0295 | Val Loss:   0.0096 | Val MAE:    0.0778 | Val R²:     0.8350
  → Mejor modelo guardado
Epoch 09 | Train Loss: 0.0278 | Val Loss:   0.0100 | Val MAE:    0.0825 | Val R²:     0.8217
Epoch 10 | Train Loss: 0.0258 | 

In [7]:
# ─── MÓDULO G.1 (versión con validación) ───

import torch.optim as optim
from torch.amp import autocast

# 1. Restaurar checkpoint
ckpt = torch.load('best_checkpoint.pt', map_location=device)
model.load_state_dict(ckpt['model_state'])
optimizer.load_state_dict(ckpt['opt_state'])
scaler.load_state_dict(ckpt['scaler_state'])

# 2. Descongelar layer3 + layer4
for name, param in model.backbone.named_parameters():
    if 'layer3' in name or 'layer4' in name:
        param.requires_grad = True

# 3. Reconfigurar optimizador
optimizer = optim.Adam([
    {'params': model.backbone.layer3.parameters(), 'lr': 5e-5},
    {'params': model.backbone.layer4.parameters(), 'lr': 1e-4},
    {'params': model.meta_net.parameters(),           'lr': 1e-3},
    {'params': model.head.parameters(),               'lr': 1e-3},
])

# 4. Fine-tuning con validación
fine_epochs = 10
for epoch in range(1, fine_epochs + 1):
    # --- Entrenamiento ---
    model.train()
    for imgs, metas, labels in train_loader:
        imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast(device_str):
            loss = loss_fn(model(imgs, metas).squeeze(1), labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    # --- Validación ---
    model.eval()
    val_losses, all_preds, all_labels = [], [], []
    with torch.no_grad():
        for imgs, metas, labels in val_loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            with autocast(device_str):
                preds = model(imgs, metas).squeeze(1)
                val_losses.append(loss_fn(preds, labels).item())
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())
    val_loss = sum(val_losses) / len(val_losses)
    y_pred   = torch.cat(all_preds).numpy()
    y_true   = torch.cat(all_labels).numpy()
    val_mae  = mean_absolute_error(y_true, y_pred)
    val_r2   = r2_score(y_true, y_pred)

    print(
        f"[Fine-Tuning] Epoch {epoch:02d} | "
        f"Val Loss: {val_loss:.4f} | "
        f"Val MAE:  {val_mae:.4f} | "
        f"Val R²:   {val_r2:.4f}"
    )

print("→ Fine-tuning de layer3+4 completado")



[Fine-Tuning] Epoch 01 | Val Loss: 0.0216 | Val MAE:  0.1277 | Val R²:   0.5863
[Fine-Tuning] Epoch 02 | Val Loss: 0.0099 | Val MAE:  0.0841 | Val R²:   0.8126
[Fine-Tuning] Epoch 03 | Val Loss: 0.0173 | Val MAE:  0.1073 | Val R²:   0.7165
[Fine-Tuning] Epoch 04 | Val Loss: 0.0116 | Val MAE:  0.0806 | Val R²:   0.8134
[Fine-Tuning] Epoch 05 | Val Loss: 0.0114 | Val MAE:  0.0850 | Val R²:   0.8109
[Fine-Tuning] Epoch 06 | Val Loss: 0.0041 | Val MAE:  0.0559 | Val R²:   0.9198
[Fine-Tuning] Epoch 07 | Val Loss: 0.0044 | Val MAE:  0.0568 | Val R²:   0.9065
[Fine-Tuning] Epoch 08 | Val Loss: 0.0068 | Val MAE:  0.0707 | Val R²:   0.8597
[Fine-Tuning] Epoch 09 | Val Loss: 0.0025 | Val MAE:  0.0422 | Val R²:   0.9494
[Fine-Tuning] Epoch 10 | Val Loss: 0.0187 | Val MAE:  0.1196 | Val R²:   0.6908
→ Fine-tuning de layer3+4 completado


In [9]:
#!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 21.7 MB/s eta 0:00:00


In [10]:
# ─── MÓDULO G.2: BÚSQUEDA DE HYPERPARÁMETROS CON OPTUNA ───

import optuna
import torch.optim as optim
from torch.amp import autocast, GradScaler
from sklearn.metrics import mean_absolute_error

# 1. Función objetivo
def objective(trial):
    # Hiparams a explorar
    lr_back3 = trial.suggest_loguniform('lr_back3', 1e-5, 1e-4)
    lr_back4 = trial.suggest_loguniform('lr_back4', 1e-5, 1e-4)
    lr_meta  = trial.suggest_loguniform('lr_meta',  1e-4, 1e-2)
    head1    = trial.suggest_categorical('head1',    [128, 256, 512])
    head2    = trial.suggest_categorical('head2',    [64, 128, 256])
    drop_p   = trial.suggest_uniform('dropout',      0.1, 0.5)

    # Reconstruir modelo
    model = DistanceModel(
        num_meta_feat=num_meta,
        head_sizes=[head1, head2],
        backbone_name='resnet50'
    ).to(device)
    # Ajustar dropout dinámico
    for m in model.meta_net:
        if isinstance(m, nn.Dropout):
            m.p = drop_p
    for m in model.head:
        if isinstance(m, nn.Dropout):
            m.p = drop_p

    # Congelar todo menos layer3+4 inicialmente
    for name, param in model.backbone.named_parameters():
        param.requires_grad = False
        if 'layer3' in name or 'layer4' in name:
            param.requires_grad = True

    # Optimizador con tasas diferenciadas
    optimizer = optim.Adam([
        {'params': model.backbone.layer3.parameters(), 'lr': lr_back3},
        {'params': model.backbone.layer4.parameters(), 'lr': lr_back4},
        {'params': model.meta_net.parameters(),           'lr': lr_meta},
        {'params': model.head.parameters(),               'lr': lr_meta},
    ])
    scaler = GradScaler(device_str)

    # Entrenar 3 epochs rápidos
    model.train()
    for _ in range(3):
        for imgs, metas, labels in train_loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            optimizer.zero_grad()
            with autocast(device_str):
                preds = model(imgs, metas).squeeze(1)
                loss  = nn.MSELoss()(preds, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

    # Validación MAE
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, metas, labels in val_loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            preds = model(imgs, metas).squeeze(1)
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())
    y_pred = torch.cat(all_preds).numpy()
    y_true = torch.cat(all_labels).numpy()
    val_mae = mean_absolute_error(y_true, y_pred)
    return val_mae

# 2. Lanzar estudio
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

print("▶️ Mejores parámetros:", study.best_params)
print("▶️ Mejor MAE:",      study.best_value)


[I 2025-05-26 00:08:09,149] A new study created in memory with name: no-name-23562622-1404-4763-9133-4fcadfeff2e8
<ipython-input-10-11e71c820742>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr_back3 = trial.suggest_loguniform('lr_back3', 1e-5, 1e-4)
<ipython-input-10-11e71c820742>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr_back4 = trial.suggest_loguniform('lr_back4', 1e-5, 1e-4)
<ipython-input-10-11e71c820742>:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr_meta  = trial.suggest_loguniform('

▶️ Mejores parámetros: {'lr_back3': 9.640041646656855e-05, 'lr_back4': 9.688376738562348e-05, 'lr_meta': 0.00010528597139765888, 'head1': 128, 'head2': 128, 'dropout': 0.33870518188826954}
▶️ Mejor MAE: 0.07563810795545578


In [11]:
# ─── MÓDULO H: ENTRENAMIENTO FINAL CON BEST_PARAMS ───

import torch.optim as optim
from torch.amp import autocast, GradScaler

# 1. Extraer mejores hiperparámetros
bp = study.best_params
lr_back3 = bp['lr_back3']
lr_back4 = bp['lr_back4']
lr_meta  = bp['lr_meta']
head1    = bp['head1']
head2    = bp['head2']
drop_p   = bp['dropout']

# 2. Reconstruir modelo con best head sizes y dropout
model = DistanceModel(
    num_meta_feat=num_meta,
    head_sizes=[head1, head2],
    backbone_name='resnet50'
).to(device)

# Ajustar dropout dinámico
for m in model.meta_net:
    if isinstance(m, nn.Dropout):
        m.p = drop_p
for m in model.head:
    if isinstance(m, nn.Dropout):
        m.p = drop_p

# 3. Congelar todo menos layer3+4
for name, param in model.backbone.named_parameters():
    param.requires_grad = False
    if 'layer3' in name or 'layer4' in name:
        param.requires_grad = True

# 4. Optimizer con tasas diferenciadas
optimizer = optim.Adam([
    {'params': model.backbone.layer3.parameters(), 'lr': lr_back3},
    {'params': model.backbone.layer4.parameters(), 'lr': lr_back4},
    {'params': model.meta_net.parameters(),           'lr': lr_meta},
    {'params': model.head.parameters(),               'lr': lr_meta},
])

# 5. Escaler y loss
scaler     = GradScaler(device_str)
loss_fn    = nn.MSELoss()
num_epochs = 30
patience   = 7
best_val   = float('inf')
stale      = 0

# 6. Loop completo
for epoch in range(1, num_epochs + 1):
    # Training
    model.train()
    train_losses = []
    for imgs, metas, labels in train_loader:
        imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast(device_str):
            preds = model(imgs, metas).squeeze(1)
            loss  = loss_fn(preds, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_losses.append(loss.item())
    # Validation
    model.eval()
    val_losses, preds, trues = [], [], []
    with torch.no_grad():
        for imgs, metas, labels in val_loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            with autocast(device_str):
                out = model(imgs, metas).squeeze(1)
                val_losses.append(loss_fn(out, labels).item())
            preds.append(out.cpu())
            trues.append(labels.cpu())
    # Métricas
    val_loss = sum(val_losses) / len(val_losses)
    y_pred   = torch.cat(preds).numpy()
    y_true   = torch.cat(trues).numpy()
    val_mae  = mean_absolute_error(y_true, y_pred)
    val_r2   = r2_score(y_true, y_pred)
    print(f"Epoch {epoch:02d} | Train Loss: {np.mean(train_losses):.4f} | Val MAE: {val_mae:.4f} | Val R²: {val_r2:.4f}")

    # Early stopping simple
    if val_loss < best_val:
        best_val = val_loss
        stale    = 0
        torch.save(model.state_dict(), 'final_model.pt')
    else:
        stale += 1
        if stale >= patience:
            print(f"No mejora en {patience} epochs. Deteniendo.")
            break

print("Entrenamiento finalizado. Modelo guardado en final_model.pt")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 01 | Train Loss: 0.1075 | Val MAE: 0.1561 | Val R²: 0.3410
Epoch 02 | Train Loss: 0.0440 | Val MAE: 0.0915 | Val R²: 0.7691
Epoch 03 | Train Loss: 0.0408 | Val MAE: 0.1145 | Val R²: 0.6750
Epoch 04 | Train Loss: 0.0379 | Val MAE: 0.1622 | Val R²: 0.2370
Epoch 05 | Train Loss: 0.0257 | Val MAE: 0.0821 | Val R²: 0.8288
Epoch 06 | Train Loss: 0.0237 | Val MAE: 0.0831 | Val R²: 0.8252
Epoch 07 | Train Loss: 0.0235 | Val MAE: 0.0543 | Val R²: 0.9180
Epoch 08 | Train Loss: 0.0204 | Val MAE: 0.0457 | Val R²: 0.9402
Epoch 09 | Train Loss: 0.0203 | Val MAE: 0.0496 | Val R²: 0.9299
Epoch 10 | Train Loss: 0.0197 | Val MAE: 0.0647 | Val R²: 0.8917
Epoch 11 | Train Loss: 0.0172 | Val MAE: 0.0785 | Val R²: 0.8214
Epoch 12 | Train Loss: 0.0176 | Val MAE: 0.0443 | Val R²: 0.9461
Epoch 13 | Train Loss: 0.0148 | Val MAE: 0.0598 | Val R²: 0.9103
Epoch 14 | Train Loss: 0.0157 | Val MAE: 0.0594 | Val R²: 0.9007
Epoch 15 | Train Loss: 0.0136 | Val MAE: 0.0544 | Val R²: 0.9189
No mejora en 7 epochs. De